In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time

symbols = ["H","Be","H"]
geometry = np.array([ -1.3/0.529, 0.0, 0.0,0.0, 0.0, 0.0, 1.3/0.529, 0.0, 0.0])

In [2]:
H, qubits = qchem.molecular_hamiltonian(
    symbols,
    geometry,
    active_electrons=4,
    active_orbitals=6
)

active_electrons = 4

singles, doubles = qchem.excitations(active_electrons, qubits)

print(f"Total number of excitations = {len(singles) + len(doubles)}")

Total number of excitations = 92


In [3]:
hf_state = qchem.hf_state(active_electrons, qubits)


def circuit_1(params, excitations):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)
    return qml.expval(H)

In [4]:
dev = qml.device("default.qubit", wires=qubits)
cost_fn = qml.QNode(circuit_1, dev)

circuit_gradient = qml.grad(cost_fn, argnum=0)

params = [0.0] * len(doubles)
grads = circuit_gradient(params, excitations=doubles)

for i in range(len(doubles)):
    print(f"Excitation : {doubles[i]}, Gradient: {grads[i]}")

Excitation : [0, 1, 4, 5], Gradient: -0.04979644496335472
Excitation : [0, 1, 4, 7], Gradient: 0.0
Excitation : [0, 1, 4, 9], Gradient: 0.0
Excitation : [0, 1, 4, 11], Gradient: 0.0
Excitation : [0, 1, 5, 6], Gradient: 0.0
Excitation : [0, 1, 5, 8], Gradient: 0.0
Excitation : [0, 1, 5, 10], Gradient: 0.0
Excitation : [0, 1, 6, 7], Gradient: -0.049796444963360664
Excitation : [0, 1, 6, 9], Gradient: 0.0
Excitation : [0, 1, 6, 11], Gradient: 0.0
Excitation : [0, 1, 7, 8], Gradient: 0.0
Excitation : [0, 1, 7, 10], Gradient: 0.0
Excitation : [0, 1, 8, 9], Gradient: -0.07705748458142693
Excitation : [0, 1, 8, 11], Gradient: 0.0
Excitation : [0, 1, 9, 10], Gradient: 0.0
Excitation : [0, 1, 10, 11], Gradient: -0.056547514341893534
Excitation : [0, 2, 4, 6], Gradient: 0.0
Excitation : [0, 2, 4, 8], Gradient: 0.0
Excitation : [0, 2, 4, 10], Gradient: 0.0
Excitation : [0, 2, 6, 8], Gradient: 0.0
Excitation : [0, 2, 6, 10], Gradient: 0.0
Excitation : [0, 2, 8, 10], Gradient: 0.011171951276909516


In [5]:
doubles_select = [doubles[i] for i in range(len(doubles)) if abs(grads[i]) > 1.0e-5]
doubles_select

[[0, 1, 4, 5],
 [0, 1, 6, 7],
 [0, 1, 8, 9],
 [0, 1, 10, 11],
 [0, 2, 8, 10],
 [0, 3, 8, 11],
 [0, 3, 9, 10],
 [1, 2, 8, 11],
 [1, 2, 9, 10],
 [1, 3, 9, 11],
 [2, 3, 4, 5],
 [2, 3, 6, 7],
 [2, 3, 8, 9],
 [2, 3, 10, 11]]

In [6]:
opt = qml.GradientDescentOptimizer(stepsize=0.5)

params_doubles = np.zeros(len(doubles_select), requires_grad=True)

for n in range(20):
    params_doubles = opt.step(cost_fn, params_doubles, excitations=doubles_select)

In [7]:
def circuit_2(params, excitations, gates_select, params_select):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, gate in enumerate(gates_select):
        if len(gate) == 4:
            qml.DoubleExcitation(params_select[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params_select[i], wires=gate)

    for i, gate in enumerate(excitations):
        if len(gate) == 4:
            qml.DoubleExcitation(params[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params[i], wires=gate)
    return qml.expval(H)

In [8]:
cost_fn = qml.QNode(circuit_2, dev)
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(singles)

grads = circuit_gradient(
    params,
    excitations=singles,
    gates_select=doubles_select,
    params_select=params_doubles
)

for i in range(len(singles)):
    print(f"Excitation : {singles[i]}, Gradient: {grads[i]}")

Excitation : [0, 4], Gradient: 0.0
Excitation : [0, 6], Gradient: 0.0
Excitation : [0, 8], Gradient: -0.0065945351897329745
Excitation : [0, 10], Gradient: 0.0
Excitation : [1, 5], Gradient: 0.0
Excitation : [1, 7], Gradient: 0.0
Excitation : [1, 9], Gradient: 0.006594533076260982
Excitation : [1, 11], Gradient: 0.0
Excitation : [2, 4], Gradient: 0.0
Excitation : [2, 6], Gradient: 0.0
Excitation : [2, 8], Gradient: 0.0
Excitation : [2, 10], Gradient: 0.004283344654791644
Excitation : [3, 5], Gradient: 0.0
Excitation : [3, 7], Gradient: 0.0
Excitation : [3, 9], Gradient: 0.0
Excitation : [3, 11], Gradient: -0.004283340181110242


In [9]:
singles_select = [singles[i] for i in range(len(singles)) if abs(grads[i]) > 1.0e-5 ]
singles_select

[[0, 8], [1, 9], [2, 10], [3, 11]]

In [10]:
cost_fn = qml.QNode(circuit_1, dev)

params = np.zeros(len(doubles_select + singles_select), requires_grad=True)

gates_select = doubles_select + singles_select

for n in range(20):
    t1 = time.time()
    params, energy = opt.step_and_cost(cost_fn, params, excitations=gates_select)
    t2 = time.time()
    print("n = {:},  E = {:.8f} H, t = {:.2f} s".format(n, energy, t2 - t1))

n = 0,  E = -15.56127101 H, t = 2.01 s
n = 1,  E = -15.58015548 H, t = 2.00 s
n = 2,  E = -15.58786899 H, t = 2.10 s
n = 3,  E = -15.59121019 H, t = 2.10 s
n = 4,  E = -15.59273697 H, t = 2.00 s
n = 5,  E = -15.59347197 H, t = 2.10 s
n = 6,  E = -15.59384534 H, t = 2.10 s
n = 7,  E = -15.59404600 H, t = 2.00 s
n = 8,  E = -15.59416023 H, t = 2.10 s
n = 9,  E = -15.59422896 H, t = 2.00 s
n = 10,  E = -15.59427241 H, t = 2.10 s
n = 11,  E = -15.59430103 H, t = 2.00 s
n = 12,  E = -15.59432049 H, t = 2.10 s
n = 13,  E = -15.59433405 H, t = 2.00 s
n = 14,  E = -15.59434365 H, t = 2.00 s
n = 15,  E = -15.59435053 H, t = 2.10 s
n = 16,  E = -15.59435551 H, t = 2.00 s
n = 17,  E = -15.59435913 H, t = 2.10 s
n = 18,  E = -15.59436177 H, t = 2.00 s
n = 19,  E = -15.59436371 H, t = 2.10 s
